In [15]:
import requests as r
from bs4 import BeautifulSoup
import pymongo
from pymongo import MongoClient
import time
from requests.exceptions import ConnectionError
import pandas as pd
import re

client = MongoClient("10.120.28.17", 27018)

#我們自己訂的db 和 collection
db = client.ptt
collect = db.pttMovie_4001toEnd

# def list_crawler(url):
#     """ This is the function for crawling urls
#         arg = "urls"
#         return = url list
#     """
#     links = []
#     res = r.get(url)
#     res.encoding = "utf-8"
#     soup = BeautifulSoup(res.text, 'lxml')
# #    print soup
#     index_links = soup.find('div', class_="r-list-container").select('a')
#     for link in index_links:
#         links.append("http://www.ptt.cc" + link['href'])
#     time.sleep(1)
#     return links

# def get_first_nextpage(url):
#     """ This is the function to get first next page url
#         arg = "url"
#         return = first next page's url
#     """
#     first_index = ""
#     res = r.get(url)
#     res.encoding = "utf-8"
#     soup = BeautifulSoup(res.text, 'lxml')
#     first_index = (soup.find(class_="btn-group-paging").find_all(class_="btn")[1]['href'])
#     first = int(first_index.split('/')[3].split('.')[0].replace("index", ""))
#     first_page = "index" + str(first + 1)
#     return first_page

def content_crawler(url):
    """ This is the function to get content from the list url
        arg = "url"
        return = content[
            [[tag, userid, content], [tag, userid, content], ... , [tag, userid, content]],
            count,
            author,
            title,
            publishdate,
            body
        ]      
    """
    content = []
    failed_urls = []
    try :
        res = r.get(url)
        if res.status_code == 400 or res.status_code == 404:
            print "==="
            content.append(url)
        else:
            res.encoding = "utf-8"
            soup = BeautifulSoup(res.text, 'lxml')

            pushes = get_pushes(soup)
            content.append(pushes[0:-1])
            content.append(pushes[-1])
        #    print content

            basic_info = basic_information(soup)
            content.append(basic_info[0])
            content.append(basic_info[1])
            content.append(basic_info[2])

            body = content_inner_text(soup)
            content.append(body)

        #     print "333"
        #     print content
            time.sleep(3)
        return content
    except ConnectionError as e:
        failed_urls.append(url)
        return failed_urls

def content_inner_text(soup):
    """ This is the function to get content body
        arg = soup
        return = "content body"
    """
    body = ""
    soup1 = soup.find(id="main-container")
    [s.extract() for s in soup1('span')]
    body = soup1.text.split('--')[0]
    return body

def basic_information(soup):
    """ This is the function to get title, author, publish date for every article
          arg = soup
          return = [ title, author, date]
    """
    basic_info = []
    
    author = "no author"
    title = "no title"
    datetime = "no publish date"
    
    distinguish_author = '作者'.decode("utf8")
    distinguish_title = '標題'.decode("utf8")
    distinguish_datetime = '時間'.decode("utf8")
    
    title_content = soup.find_all('span', class_="article-meta-value")
    
#    print title_content
#    print len(title_content)
    
    if len(title_content) == 4:
        author = title_content[0].text
        title = title_content[2].text
        datetime =  title_content[3].text
        basic_info.append(author)
        basic_info.append(title)
        basic_info.append(datetime)
    elif len(title_content) == 0:
        basic_info.append(author)
        basic_info.append(title)
        basic_info.append(datetime)
    else:
        metaline = soup.find_all('div', class_="article-metaline")
        print metaline
        for meta in metaline:
#            print meta.text
            if re.search(distinguish_author, meta.text):
                print meta.text
                author = meta.text.replace(distinguish_author, "")
            elif re.search(distinguish_title, meta.text):
                title = meta.text.replace(distinguish_title, "")
            elif re.search(distinguish_datetime, meta.text):
                datetime = meta.text.replace(distinguish_datetime, "")                
        basic_info.append(author)
        basic_info.append(title)
        basic_info.append(datetime)                
#        print basic_info    
    return basic_info

def get_pushes(soup):
    """ This is the crawler for each article
          arg = soup
          return = [
              [tag, userid, content],
              [tag, userid, content],
              ......,
              count
          ]
    """
    total_pushes = []
    count = 1
#    print soup
    all_pushes = soup.find_all('div', class_="push")
    
    for each_push in all_pushes:
        push = []
#        print each_push
        try:
            f1_tag = each_push.find('span', class_="f1 hl push-tag")
            h1_tag = each_push.find('span', class_="hl push-tag")    
            if f1_tag is not None:
    #            print count
                push_tag = f1_tag
            else:
    #            print "===" + str(count)
                push_tag = h1_tag
    #        print push_tag.text

            userid = each_push.find(class_="f3 hl push-userid").text
            content = each_push.find(class_="f3 push-content").text.split(":", 1)[1]

            push.append(push_tag.text)
            push.append(userid)
            push.append(content)
            
            count += 1
            total_pushes.append(push)
        except:
            continue
    
    total_pushes.append(count)
#    print total_pushes
    return total_pushes

def PTT_crawler(name):
    """ This is the main function for the ptt_crawler
          arg = neme #The board name you want to crawle
          return = "Crawler Finished"
          OutPut dict to MongoDB
    """
    count = 1
    failure_count = 1
    failure_lists = []
    url = "https://www.ptt.cc/bbs/" + name + "/index.html"
    
    
    # get the latest page number and change it into int i for counting
#    first_page = get_first_nextpage(url)
#    i = int(first_page.replace("index", ""))
     
    i = 4533
    
    #    while ( i >= 0):
    while ( i > 4000): # if you don't want to crawl all the page set cutoff here
        
        print i # check which page we've crawl

        # get pageList
        page_list = list_crawler("https://www.ptt.cc/bbs/" + name + "/index" + str(i) + ".html")
        i -= 1
                
        for links in page_list:
            
            article = {}
            
            # call content_crawler() function get a list of [[[tag, userid, content], [tag, userid, content], ... , [tag, userid, content]], count, author, title, publishdate, body] 
            content = content_crawler(links)

            if len(content) > 1:
                article.update({"author" : content[-4]})
                article.update({"title" : content[-3]})
                article.update({"publishdate" : content[-2]})
                article.update({"body" : content[-1]})
                article.update({"ttlpush" : content[0:-5]})
                article.update({"count" : content[-5]})
                article.update({"url" : links})
            
                collect.insert_one(article)
                count += 1
                now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                print now + '\t' + content[-3] + '\n' + links
#                print count
#                print article
            else:                
                failure_lists.append(content[0])
                failure_count += 1
                now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                print "Fail!" + now + '\t' + content[0]
    fail_url = pd.DataFrame(failure_lists)     
    filename = 'PTT_fail.csv'
    fail_url.to_csv(filename, encoding='utf8')
    
    return "Crawler Finished!  Total articles number =  " + str(count-1) + "   Failure number = " + str(failure_count-1)

In [16]:
PTT_crawler("movie")

4533
2016-11-01 08:11:03	[片單] 求推適合闔家觀賞的院線片
http://www.ptt.cc/bbs/movie/M.1473329705.A.D8F.html
2016-11-01 08:11:08	[翻譯] 誠實預告片--蝙蝠俠：致命玩笑
http://www.ptt.cc/bbs/movie/M.1473330143.A.4BA.html
2016-11-01 08:11:13	[新聞] 詹姆斯卡麥隆透露《阿凡達2》的故事方向
http://www.ptt.cc/bbs/movie/M.1473331984.A.306.html
2016-11-01 08:11:19	[請益] 屍速列車票房走勢
http://www.ptt.cc/bbs/movie/M.1473332195.A.A74.html
2016-11-01 08:11:29	[一般雷跟負雷] 屍速列車&銷售奇姬
http://www.ptt.cc/bbs/movie/M.1473333323.A.331.html
2016-11-01 08:11:35	[好雷] 薩利機長：哈德遜奇蹟
http://www.ptt.cc/bbs/movie/M.1473334267.A.B62.html
2016-11-01 08:11:41	[情報] 《屍速列車》新加坡、香港再創新紀錄
http://www.ptt.cc/bbs/movie/M.1473334401.A.04B.html
2016-11-01 08:11:46	[討論] 屍速列車若用V8拍攝 是否會更好看？
http://www.ptt.cc/bbs/movie/M.1473335969.A.8DE.html
2016-11-01 08:11:52	[微好雷]《美味秘方》，料理的原點是謂家。 
http://www.ptt.cc/bbs/movie/M.1473336816.A.A9F.html
2016-11-01 08:11:57	[情報] ID4星際重生 二輪上映
http://www.ptt.cc/bbs/movie/M.1473336978.A.5CB.html
2016-11-01 08:12:03	[問片] 一部跟消防員有關的韓國電影
http://www.ptt.cc/bbs/movie/M.14733

'Crawler Finished!  Total articles number =  10649   Failure number = 9'